In [1]:
# %%
import torch
import timm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# %%
# ================================
# 1️⃣ LOAD DATASET
# ================================
dataset_name = "Team-SknAI/SknAI_300_v3_11Labels"
datasets = load_dataset(dataset_name)
datasets = datasets["train"].train_test_split(test_size=0.2, stratify_by_column="label")

In [2]:
# ================================
# 2️⃣ DEFINE TRANSFORMATIONS
# ================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================================
# 3️⃣ CUSTOM DATASET CLASS
# ================================
class SkinDataset(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        try:
            img = self.dataset[idx]["image"]
            img = Image.fromarray(np.array(img).astype("uint8"))
            img = img.convert("RGB")
            img = self.transform(img)
            label = torch.tensor(self.dataset[idx]["label"], dtype=torch.long)
            return img, label
        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
            return torch.zeros(3, 224, 224), torch.tensor(0)

# ================================
# 4️⃣ CREATE DATA LOADERS
# ================================
train_dataset = SkinDataset(datasets["train"], transform)
val_dataset = SkinDataset(datasets["test"], transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)



In [3]:
# ================================
# 5️⃣ DEFINE MODEL
# ================================
class DenseNet121Model(nn.Module):
    def __init__(self, num_classes):
        super(DenseNet121Model, self).__init__()
        self.model = timm.create_model("densenet121", pretrained=True, num_classes=num_classes)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)

num_classes = len(datasets["train"].features["label"].names)
model = DenseNet121Model(num_classes)


# %%
# ================================
# 6️⃣ TRAINING SETUP
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)


In [4]:
# ================================
# 7️⃣ TRAINING LOOP WITH TRACKING
# ================================
EPOCHS = 15
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()
        total += labels.size(0)
    
    scheduler.step()
    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {running_loss:.4f}, Train Accuracy: {train_accuracies[-1]:.2f}%")



c:\Users\vivek\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/15 - Loss: 51.1400, Train Accuracy: 61.55%
Epoch 2/15 - Loss: 18.3476, Train Accuracy: 85.57%
Epoch 3/15 - Loss: 11.8029, Train Accuracy: 91.59%
Epoch 4/15 - Loss: 6.3409, Train Accuracy: 95.68%
Epoch 5/15 - Loss: 3.4451, Train Accuracy: 97.39%
Epoch 6/15 - Loss: 2.1885, Train Accuracy: 98.45%
Epoch 7/15 - Loss: 1.6309, Train Accuracy: 98.45%
Epoch 8/15 - Loss: 1.0640, Train Accuracy: 99.28%
Epoch 9/15 - Loss: 0.9395, Train Accuracy: 99.39%
Epoch 10/15 - Loss: 0.6009, Train Accuracy: 99.58%
Epoch 11/15 - Loss: 0.6882, Train Accuracy: 99.62%
Epoch 12/15 - Loss: 0.7937, Train Accuracy: 99.58%
Epoch 13/15 - Loss: 0.5672, Train Accuracy: 99.62%
Epoch 14/15 - Loss: 1.0030, Train Accuracy: 99.58%
Epoch 15/15 - Loss: 0.8395, Train Accuracy: 99.51%


In [5]:
# ================================
# 8️⃣ EVALUATION & METRICS
# ================================
model.eval()
correct, total = 0, 0
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

val_accuracy = 100 * correct / total
val_accuracies.append(val_accuracy)
print(f"Test Accuracy: {val_accuracy:.2f}%")

Test Accuracy: 90.00%
